In [1]:
%gui qt5
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
from matplotlib import pyplot as plt
from matplotlib.widgets import MultiCursor
import numpy as np

In [4]:
from theplotter.graph import nodes
from theplotter.plugins.targets import mpl

In [79]:
class NpArraySource(nodes.Source):
    def __init__(self, arr, **kwargs):
        super().__init__(**kwargs)
        self.dataset = arr
        
    def run(self):
        self.need_update = False
        

class MultiArray(nodes.Node):
    def __init__(self, **arrays):
        super().__init__()
        
        self.names = []
        for k, v in arrays.items():
            src = NpArraySource(v)
            self.datasources.append(src)
            src.add_target_if_absent(self)
            self.names.append(k)
            
    def set_array(self, name, value):
        src = self.datasources[self.names.index(name)]
        src.dataset = value
        self.need_update = True
            
    def __getitem__(self, item):
        return self.datasources[self.names.index(item)]
    
    def __setitem__ (self, key, item):
        self.set_array(key, item)

In [80]:
x = np.linspace(0,10,101)
z = np.cos(x)

In [81]:
xsrc = NpArraySource(x)
zsrc = NpArraySource(z)

In [82]:
xzsrc = MultiArray(x=x, z=z)

In [83]:
w = mpl.MPLWindow(1,1)
plt = mpl.lineplot(datasources=[xzsrc['x'], xzsrc['z']], plottarget=w[0,0])

In [84]:
xzsrc['z'] = z**2

True

In [41]:
zsrc.dataset = z**2
zsrc.need_update = True
# plt.play()
# zsrc.run()

In [43]:
lst = ['a', 'b']

In [45]:
lst.index('a')

0